# Modeling

In [1]:
import numpy as np
import pandas as pd

from pprint import pprint

import unicodedata

import re

import json

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

import acquire as ac
import prepare as pr
import preprocessing as pp

---
## Wrangle

In [2]:
df = pr.wrangle_readme_data()
df

,repo,language,readme_contents,clean_readme_contents,len_of_clean_readme_contents
0,rdpeng/ProgrammingAssignment2,R,### Introduction\n\nThis second programming as...,introduction second programming assignment req...,332
1,octocat/Spoon-Knife,HTML,### Well hello there!\n\nThis repository is me...,well hello repository meant provide example fo...,62
2,tensorflow/tensorflow,C++,"<div align=""center"">\n <img src=""https://www....",div aligncenter srchttpswwwtensorfloworgimages...,434
3,SmartThingsCommunity/SmartThingsPublic,Groovy,# SmartThings Public GitHub Repo\n\nAn officia...,smartthings public github repo official list s...,32
4,twbs/bootstrap,JavaScript,"<p align=""center"">\n <a href=""https://getboot...",p aligncenter hrefhttpsgetbootstrapcom srchttp...,567
...,...,...,...,...,...
214,akveo/ngx-admin,TypeScript,"# ngx-admin [<img src=""https://i.imgur.com/oMc...",ngxadmin srchttpsiimgurcomomcxwz0png alteva de...,300
215,swirldev/swirl_courses,R,# swirl courses\n\nThis is a collection of int...,swirl course collection interactive course use...,393
216,jrowberg/i2cdevlib,C++,Jennic platform added!\n\n====================...,jennic platform added note detail project plea...,250
217,etcd-io/etcd,Go,# etcd\n\n[![Go Report Card](https://goreportc...,etcd go report cardhttpsgoreportcardcombadgegi...,630


In [3]:
df.columns

Index(['repo', 'language', 'readme_contents', 'clean_readme_contents',
       'len_of_clean_readme_contents'],
      dtype='object')

In [4]:
df.repo.value_counts().head(18)

SmartThingsCommunity/SmartThingsPublic                                         2
nightscout/cgm-remote-monitor                                                  2
rust-lang/rust                                                                 2
fengdu78/Coursera-ML-AndrewNg-Notes                                            2
arduino/Arduino                                                                2
apache/incubator-mxnet                                                         2
axios/axios                                                                    2
CamDavidsonPilon/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers    2
twbs/bootstrap                                                                 2
woocommerce/woocommerce                                                        2
octocat/Spoon-Knife                                                            2
rdpeng/ProgrammingAssignment2                                                  2
tensorflow/models           

In [5]:
df.repo.value_counts().tail()

zxing/zxing              1
kubernetes/kubernetes    1
microsoft/vscode         1
vinta/awesome-python     1
bcit-ci/CodeIgniter      1
Name: repo, dtype: int64

In [6]:
len(df.repo.unique())

202

In [7]:
# stratification base on language using train_test_split won't work unless we have more than one observation
# per language
df.language.value_counts()

JavaScript          45
Java                35
Python              29
C++                 18
HTML                16
Go                   9
Jupyter Notebook     9
PHP                  9
Ruby                 7
TypeScript           7
CSS                  6
C                    5
Vue                  3
R                    3
Rust                 3
PowerShell           2
C#                   2
Groovy               2
Kotlin               2
Shell                2
Objective-C          1
Scala                1
ApacheConf           1
Swift                1
TeX                  1
Name: language, dtype: int64

---
## Model

---
### Outlier Detour

In [8]:
# lemmas
# list_of_readmes = df.clean_readme_contents.tolist()
# list_of_readmes

In [9]:
# for index in range(len(list_of_readmes)):
#     list_of_readmes[index] = list_of_readmes[index].split()
    

# print(list_of_readmes)

In [10]:
# list_of_list_of_lemmas = [lemma.split() for lemmas in list_of_lemmas]
# list_of_list_of_lemmas

In [11]:
# len(list_of_readmes[0])

In [12]:
# len(list_of_readmes[-1])

In [13]:
# len_of_readmes = [len(readme) for readme in list_of_readmes]
# len_of_readmes

In [14]:
# len_of_readmes.sort()

In [15]:
# print(len_of_readmes)

**Cutoff for length of words in readme = 10**

In [16]:
index_three_list = df[df.index == 3].clean_readme_contents.values.tolist()
index_three_list

['smartthings public github repo official list smartapps device type smartthings link help get started coding right away githubspecific documentationhttpdocssmartthingscomenlatesttoolsandidegithubintegrationhtml full documentationhttpdocssmartthingscom ide simulatorhttpidesmartthingscom community forumshttpcommunitysmartthingscom follow u web twitter httptwittercomsmartthingsdev facebook httpfacebookcomsmartthingsdevelopers']

In [17]:
df[df.index == 3].len_of_clean_readme_contents

3    32
Name: len_of_clean_readme_contents, dtype: int64

**len_of_clean_readme_contents seems to be working**

In [18]:
df.len_of_clean_readme_contents.value_counts().sort_index()

10       1
13       2
14       2
30       2
32       2
        ..
5682     1
7278     1
8039     1
10293    1
16635    1
Name: len_of_clean_readme_contents, Length: 182, dtype: int64

In [19]:
df.shape

(219, 5)

**225 observations before outlier removal; should have 219 after**

**CHECK**

---
### Back on Model Track

#### Train/Test Split

In [20]:
df = pr.wrangle_readme_data()
train, test = pp.split_repo_data(df)

In [21]:
print(train.shape[0]/df.shape[0])

0.7990867579908676


In [22]:
print(test.shape[0]/df.shape[0])

0.2009132420091324
